<a href="https://colab.research.google.com/github/skywalker0803r/Ricky/blob/master/%E7%8E%89%E5%B1%B1%E9%8A%80%E8%A1%8CAI%E6%AF%94%E8%B3%BD_featuresengineering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
import gc
import warnings 
warnings.simplefilter('ignore')
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# 中文map

In [0]:
中文map = {'bacno':'歸戶帳號','txkey':'交易序號','locdt':'授權日期','loctm':'授權時間','cano':'交易卡號',
         'contp':'交易類別','etymd':'交易型態','mchno':'特店代號','acqic':'收單行代碼','mcc':'MCC_CODE',
         'conam':'交易金額-台幣(經過轉換)','ecfg':'網路交易註記','insfg':'分期交易註記','iterm':'分期期數',
         'stocn':'消費地國別','scity':'消費城市','stscd':'狀態碼','ovrlt':'超額註記碼','flbmk':'Fallback註記',
         'hcefg':'支付型態','csmcu':'消費地幣別','flg_3dsmk':'3DS交易註記','fraud_ind':'盜刷註記'}

# train data

In [3]:
import pandas as pd
train = pd.read_csv("/content/drive/My Drive/玉山人工智慧比賽數據/train_預處理完.csv",index_col=0)
train_fraud_ind = train['fraud_ind']
train_txkey = train['txkey']
print(train.shape)
train.head()

(1521787, 23)


,acqic,bacno,cano,conam,contp,csmcu,ecfg,etymd,flbmk,flg_3dsmk,hcefg,insfg,iterm,locdt,loctm,mcc,mchno,ovrlt,scity,stocn,stscd,txkey,fraud_ind
0,6862,112785,37846,16158,5,0,0,0,0,0,5,0,0,33,61954,457,59034,0,0,102,0,516056,0
1,0,133951,45476,13693,5,0,0,2,0,0,0,0,0,9,38216,451,0,0,5795,102,0,4376,0
2,6862,15350,187354,16158,5,0,0,0,0,0,5,0,0,6,54640,457,59034,0,0,102,0,483434,0
3,6697,156492,29812,40413,5,62,0,5,0,0,5,0,0,5,62128,247,50185,0,3267,102,0,1407164,0
4,5959,105534,80881,25962,5,62,0,4,0,0,5,0,0,6,65231,263,93290,0,5795,102,0,1051004,0


# test data

In [4]:
test = pd.read_csv("/content/drive/My Drive/玉山人工智慧比賽數據/test_預處理完.csv",index_col=0)
test_txkey = test['txkey']
print(test.shape)
test.head()

(421665, 22)


,acqic,bacno,cano,conam,contp,csmcu,ecfg,etymd,flbmk,flg_3dsmk,hcefg,insfg,iterm,locdt,loctm,mcc,mchno,ovrlt,scity,stocn,stscd,txkey
0,6862,162489,116168,16158,5,0,0,0,0,0,5,0,0,102,77950,457,59060,0,0,102,0,592489
1,6862,162489,116168,16158,5,0,0,0,0,0,5,0,0,102,79549,457,59060,0,0,102,0,592452
2,6862,162489,116168,16158,5,0,0,0,0,0,5,0,0,100,60355,457,59060,0,0,102,0,590212
3,6862,162489,116168,16158,5,0,0,0,0,0,5,0,0,100,60296,457,59060,0,0,102,0,590209
4,6862,162489,116168,16158,5,0,0,0,0,0,5,0,0,102,77933,457,59060,0,0,102,0,592488


# full data

In [5]:
full_data  = train.drop('fraud_ind',axis=1).append(test)
print(full_data.shape)
full_data.head()

(1943452, 22)


,acqic,bacno,cano,conam,contp,csmcu,ecfg,etymd,flbmk,flg_3dsmk,hcefg,insfg,iterm,locdt,loctm,mcc,mchno,ovrlt,scity,stocn,stscd,txkey
0,6862,112785,37846,16158,5,0,0,0,0,0,5,0,0,33,61954,457,59034,0,0,102,0,516056
1,0,133951,45476,13693,5,0,0,2,0,0,0,0,0,9,38216,451,0,0,5795,102,0,4376
2,6862,15350,187354,16158,5,0,0,0,0,0,5,0,0,6,54640,457,59034,0,0,102,0,483434
3,6697,156492,29812,40413,5,62,0,5,0,0,5,0,0,5,62128,247,50185,0,3267,102,0,1407164
4,5959,105534,80881,25962,5,62,0,4,0,0,5,0,0,6,65231,263,93290,0,5795,102,0,1051004


In [6]:
del train
del test
gc.collect()

199

# 定義特徵 去除fraud_ind,txkey,locdt,其他都當特徵

In [7]:
features = full_data.drop(['txkey','locdt'],axis=1).columns.tolist()
print(len(features))
print(features)

20
['acqic', 'bacno', 'cano', 'conam', 'contp', 'csmcu', 'ecfg', 'etymd', 'flbmk', 'flg_3dsmk', 'hcefg', 'insfg', 'iterm', 'loctm', 'mcc', 'mchno', 'ovrlt', 'scity', 'stocn', 'stscd']


# Function for Numeric Aggregations

In [0]:
def agg_function(df,group_var,features):
    agg = df[features].groupby(group_var).agg(["count","nunique"]).reset_index()

    columns = [group_var]
    for var in agg.columns.levels[0]:
        if var != group_var:
            for stat in agg.columns.levels[1][:-1]:
                columns.append('%s_%s_%s' % (group_var,var,stat))
    agg.columns = columns
    return agg

# 用cano分組計算該cano對features的 count ,nunique

In [9]:
cano_agg = agg_function(full_data,"cano",features)
full_data = full_data.merge(cano_agg, on = "cano", how = 'left')
print(full_data.shape)
full_data.head()

(1943452, 60)


,acqic,bacno,cano,conam,contp,csmcu,ecfg,etymd,flbmk,flg_3dsmk,hcefg,insfg,iterm,locdt,loctm,mcc,mchno,ovrlt,scity,stocn,stscd,txkey,cano_acqic_count,cano_acqic_nunique,cano_bacno_count,cano_bacno_nunique,cano_conam_count,cano_conam_nunique,cano_contp_count,cano_contp_nunique,cano_csmcu_count,cano_csmcu_nunique,cano_ecfg_count,cano_ecfg_nunique,cano_etymd_count,cano_etymd_nunique,cano_flbmk_count,cano_flbmk_nunique,cano_flg_3dsmk_count,cano_flg_3dsmk_nunique,cano_hcefg_count,cano_hcefg_nunique,cano_insfg_count,cano_insfg_nunique,cano_iterm_count,cano_iterm_nunique,cano_loctm_count,cano_loctm_nunique,cano_mcc_count,cano_mcc_nunique,cano_mchno_count,cano_mchno_nunique,cano_ovrlt_count,cano_ovrlt_nunique,cano_scity_count,cano_scity_nunique,cano_stocn_count,cano_stocn_nunique,cano_stscd_count,cano_stscd_nunique
0,6862,112785,37846,16158,5,0,0,0,0,0,5,0,0,33,61954,457,59034,0,0,102,0,516056,80,18,80,1,80,51,80,3,80,3,80,2,80,5,80,1,80,1,80,2,80,1,80,1,80,80,80,21,80,33,80,1,80,13,80,5,80,1
1,0,133951,45476,13693,5,0,0,2,0,0,0,0,0,9,38216,451,0,0,5795,102,0,4376,23,7,23,1,23,16,23,1,23,2,23,2,23,4,23,1,23,2,23,2,23,2,23,2,23,23,23,8,23,11,23,1,23,4,23,1,23,1
2,6862,15350,187354,16158,5,0,0,0,0,0,5,0,0,6,54640,457,59034,0,0,102,0,483434,18,7,18,1,18,12,18,1,18,2,18,1,18,4,18,1,18,1,18,2,18,1,18,1,18,18,18,7,18,12,18,1,18,4,18,1,18,1
3,6697,156492,29812,40413,5,62,0,5,0,0,5,0,0,5,62128,247,50185,0,3267,102,0,1407164,57,7,57,1,57,48,57,2,57,1,57,2,57,4,57,1,57,2,57,1,57,2,57,2,57,57,57,12,57,25,57,1,57,7,57,1,57,1
4,5959,105534,80881,25962,5,62,0,4,0,0,5,0,0,6,65231,263,93290,0,5795,102,0,1051004,63,11,63,1,63,35,63,3,63,2,63,2,63,5,63,1,63,2,63,2,63,1,63,1,63,63,63,16,63,28,63,1,63,8,63,1,63,1


# 用bacno分組計算該bacno對features的 count ,nunique

In [10]:
bacno_agg = agg_function(full_data,"bacno",features)
full_data = full_data.merge(bacno_agg, on = "bacno", how = 'left')
print(full_data.shape)
full_data.head()

(1943452, 98)


,acqic,bacno,cano,conam,contp,csmcu,ecfg,etymd,flbmk,flg_3dsmk,hcefg,insfg,iterm,locdt,loctm,mcc,mchno,ovrlt,scity,stocn,stscd,txkey,cano_acqic_count,cano_acqic_nunique,cano_bacno_count,cano_bacno_nunique,cano_conam_count,cano_conam_nunique,cano_contp_count,cano_contp_nunique,cano_csmcu_count,cano_csmcu_nunique,cano_ecfg_count,cano_ecfg_nunique,cano_etymd_count,cano_etymd_nunique,cano_flbmk_count,cano_flbmk_nunique,cano_flg_3dsmk_count,cano_flg_3dsmk_nunique,...,cano_stscd_count,cano_stscd_nunique,bacno_acqic_count,bacno_acqic_nunique,bacno_cano_count,bacno_cano_nunique,bacno_conam_count,bacno_conam_nunique,bacno_contp_count,bacno_contp_nunique,bacno_csmcu_count,bacno_csmcu_nunique,bacno_ecfg_count,bacno_ecfg_nunique,bacno_etymd_count,bacno_etymd_nunique,bacno_flbmk_count,bacno_flbmk_nunique,bacno_flg_3dsmk_count,bacno_flg_3dsmk_nunique,bacno_hcefg_count,bacno_hcefg_nunique,bacno_insfg_count,bacno_insfg_nunique,bacno_iterm_count,bacno_iterm_nunique,bacno_loctm_count,bacno_loctm_nunique,bacno_mcc_count,bacno_mcc_nunique,bacno_mchno_count,bacno_mchno_nunique,bacno_ovrlt_count,bacno_ovrlt_nunique,bacno_scity_count,bacno_scity_nunique,bacno_stocn_count,bacno_stocn_nunique,bacno_stscd_count,bacno_stscd_nunique
0,6862,112785,37846,16158,5,0,0,0,0,0,5,0,0,33,61954,457,59034,0,0,102,0,516056,80,18,80,1,80,51,80,3,80,3,80,2,80,5,80,1,80,1,...,80,1,80,18,80,1,80,51,80,3,80,3,80,2,80,5,80,1,80,1,80,2,80,1,80,1,80,80,80,21,80,33,80,1,80,13,80,5,80,1
1,0,133951,45476,13693,5,0,0,2,0,0,0,0,0,9,38216,451,0,0,5795,102,0,4376,23,7,23,1,23,16,23,1,23,2,23,2,23,4,23,1,23,2,...,23,1,23,7,23,1,23,16,23,1,23,2,23,2,23,4,23,1,23,2,23,2,23,2,23,2,23,23,23,8,23,11,23,1,23,4,23,1,23,1
2,6862,15350,187354,16158,5,0,0,0,0,0,5,0,0,6,54640,457,59034,0,0,102,0,483434,18,7,18,1,18,12,18,1,18,2,18,1,18,4,18,1,18,1,...,18,1,18,7,18,1,18,12,18,1,18,2,18,1,18,4,18,1,18,1,18,2,18,1,18,1,18,18,18,7,18,12,18,1,18,4,18,1,18,1
3,6697,156492,29812,40413,5,62,0,5,0,0,5,0,0,5,62128,247,50185,0,3267,102,0,1407164,57,7,57,1,57,48,57,2,57,1,57,2,57,4,57,1,57,2,...,57,1,74,8,74,2,74,56,74,2,74,1,74,2,74,4,74,1,74,2,74,1,74,2,74,2,74,74,74,14,74,36,74,1,74,9,74,1,74,1
4,5959,105534,80881,25962,5,62,0,4,0,0,5,0,0,6,65231,263,93290,0,5795,102,0,1051004,63,11,63,1,63,35,63,3,63,2,63,2,63,5,63,1,63,2,...,63,1,78,11,78,2,78,43,78,3,78,2,78,2,78,5,78,1,78,2,78,2,78,1,78,1,78,78,78,17,78,35,78,1,78,8,78,1,78,1


# 用acqic分組計算該acqic對features的 count ,nunique

In [11]:
acqic_agg = agg_function(full_data,"acqic",features)
full_data = full_data.merge(acqic_agg, on = "acqic", how = 'left')
print(full_data.shape)
full_data.head()

(1943452, 136)


,acqic,bacno,cano,conam,contp,csmcu,ecfg,etymd,flbmk,flg_3dsmk,hcefg,insfg,iterm,locdt,loctm,mcc,mchno,ovrlt,scity,stocn,stscd,txkey,cano_acqic_count,cano_acqic_nunique,cano_bacno_count,cano_bacno_nunique,cano_conam_count,cano_conam_nunique,cano_contp_count,cano_contp_nunique,cano_csmcu_count,cano_csmcu_nunique,cano_ecfg_count,cano_ecfg_nunique,cano_etymd_count,cano_etymd_nunique,cano_flbmk_count,cano_flbmk_nunique,cano_flg_3dsmk_count,cano_flg_3dsmk_nunique,...,bacno_stscd_count,bacno_stscd_nunique,acqic_bacno_count,acqic_bacno_nunique,acqic_cano_count,acqic_cano_nunique,acqic_conam_count,acqic_conam_nunique,acqic_contp_count,acqic_contp_nunique,acqic_csmcu_count,acqic_csmcu_nunique,acqic_ecfg_count,acqic_ecfg_nunique,acqic_etymd_count,acqic_etymd_nunique,acqic_flbmk_count,acqic_flbmk_nunique,acqic_flg_3dsmk_count,acqic_flg_3dsmk_nunique,acqic_hcefg_count,acqic_hcefg_nunique,acqic_insfg_count,acqic_insfg_nunique,acqic_iterm_count,acqic_iterm_nunique,acqic_loctm_count,acqic_loctm_nunique,acqic_mcc_count,acqic_mcc_nunique,acqic_mchno_count,acqic_mchno_nunique,acqic_ovrlt_count,acqic_ovrlt_nunique,acqic_scity_count,acqic_scity_nunique,acqic_stocn_count,acqic_stocn_nunique,acqic_stscd_count,acqic_stscd_nunique
0,6862,112785,37846,16158,5,0,0,0,0,0,5,0,0,33,61954,457,59034,0,0,102,0,516056,80,18,80,1,80,51,80,3,80,3,80,2,80,5,80,1,80,1,...,80,1,126507,30627,126507,33807,126507,3,126507,1,126507,2,126507,1,126507,1,126507,2,126507,2,126507,3,126507,1,126507,1,126507,56166,126507,2,126507,720,126507,2,126507,1,126507,1,126507,2
1,0,133951,45476,13693,5,0,0,2,0,0,0,0,0,9,38216,451,0,0,5795,102,0,4376,23,7,23,1,23,16,23,1,23,2,23,2,23,4,23,1,23,2,...,23,1,61355,19252,61355,20139,61355,130,61355,3,61355,7,61355,1,61355,3,61355,2,61355,2,61355,6,61355,1,61355,1,61355,7858,61355,23,61355,188,61355,2,61355,2,61355,2,61355,4
2,6862,15350,187354,16158,5,0,0,0,0,0,5,0,0,6,54640,457,59034,0,0,102,0,483434,18,7,18,1,18,12,18,1,18,2,18,1,18,4,18,1,18,1,...,18,1,126507,30627,126507,33807,126507,3,126507,1,126507,2,126507,1,126507,1,126507,2,126507,2,126507,3,126507,1,126507,1,126507,56166,126507,2,126507,720,126507,2,126507,1,126507,1,126507,2
3,6697,156492,29812,40413,5,62,0,5,0,0,5,0,0,5,62128,247,50185,0,3267,102,0,1407164,57,7,57,1,57,48,57,2,57,1,57,2,57,4,57,1,57,2,...,74,1,255226,74622,255226,84353,255226,17722,255226,5,255226,2,255226,2,255226,6,255226,3,255226,3,255226,9,255226,2,255226,9,255226,63329,255226,126,255226,18121,255226,2,255226,101,255226,1,255226,3
4,5959,105534,80881,25962,5,62,0,4,0,0,5,0,0,6,65231,263,93290,0,5795,102,0,1051004,63,11,63,1,63,35,63,3,63,2,63,2,63,5,63,1,63,2,...,78,1,159855,41913,159855,46161,159855,8646,159855,4,159855,2,159855,2,159855,6,159855,3,159855,3,159855,7,159855,1,159855,1,159855,59086,159855,130,159855,10568,159855,2,159855,34,159855,1,159855,3


# 拆回train and test

In [12]:
train = full_data.loc[full_data['txkey'].isin(train_txkey),:]
train['fraud_ind'] = train_fraud_ind
print(train.shape)

(1521787, 137)


In [13]:
test = full_data.loc[full_data['txkey'].isin(test_txkey),:]
print(test.shape)

(421665, 136)


In [14]:
del full_data
gc.collect()

0

# Correlation Function

In [0]:
# Function to calculate correlations with the target for a dataframe
def target_corrs(df,feats):
  corrs = {} 
  for col in feats:
    corr = df['fraud_ind'].corr(df[col])
    corrs[col] = abs(corr)
  return pd.DataFrame(corrs,index=['corr_score']).T.sort_values(by='corr_score',ascending=False)

In [16]:
#原始特徵
corrs = target_corrs(train,features)
corrs.head(10)

,corr_score
stscd,0.279225
ecfg,0.161701
stocn,0.161405
acqic,0.083764
etymd,0.067743
loctm,0.041972
scity,0.028736
hcefg,0.026210
contp,0.023248
conam,0.022302


In [17]:
#新特徵
new_feats = list(set(train.drop('fraud_ind',axis=1).columns)^set(features))
corrs = target_corrs(train,new_feats)
corrs.head(10)

,corr_score
acqic_scity_nunique,0.212244
acqic_csmcu_nunique,0.209596
cano_stscd_nunique,0.174197
acqic_stocn_nunique,0.161676
acqic_hcefg_nunique,0.140998
bacno_stscd_nunique,0.139140
acqic_flbmk_nunique,0.124775
acqic_loctm_nunique,0.122189
acqic_bacno_nunique,0.120377
acqic_stscd_nunique,0.119250


# 篩選新特徵

In [0]:
select_feats = features+corrs.index.tolist()[:10]
train = train[select_feats+['fraud_ind']]
test = test[select_feats]

# check data

In [20]:
print(train.shape)
print(train.isnull().sum().sum())
train.head()

(1521787, 31)
0


,acqic,bacno,cano,conam,contp,csmcu,ecfg,etymd,flbmk,flg_3dsmk,hcefg,insfg,iterm,loctm,mcc,mchno,ovrlt,scity,stocn,stscd,acqic_scity_nunique,acqic_csmcu_nunique,cano_stscd_nunique,acqic_stocn_nunique,acqic_hcefg_nunique,bacno_stscd_nunique,acqic_flbmk_nunique,acqic_loctm_nunique,acqic_bacno_nunique,acqic_stscd_nunique,fraud_ind
0,6862,112785,37846,16158,5,0,0,0,0,0,5,0,0,61954,457,59034,0,0,102,0,1,2,1,1,3,1,2,56166,30627,2,0
1,0,133951,45476,13693,5,0,0,2,0,0,0,0,0,38216,451,0,0,5795,102,0,2,7,1,2,6,1,2,7858,19252,4,0
2,6862,15350,187354,16158,5,0,0,0,0,0,5,0,0,54640,457,59034,0,0,102,0,1,2,1,1,3,1,2,56166,30627,2,0
3,6697,156492,29812,40413,5,62,0,5,0,0,5,0,0,62128,247,50185,0,3267,102,0,101,2,1,1,9,1,3,63329,74622,3,0
4,5959,105534,80881,25962,5,62,0,4,0,0,5,0,0,65231,263,93290,0,5795,102,0,34,2,1,1,7,1,3,59086,41913,3,0


In [21]:
print(test.shape)
print(test.isnull().sum().sum())
test.head()

(421665, 30)
0


,acqic,bacno,cano,conam,contp,csmcu,ecfg,etymd,flbmk,flg_3dsmk,hcefg,insfg,iterm,loctm,mcc,mchno,ovrlt,scity,stocn,stscd,acqic_scity_nunique,acqic_csmcu_nunique,cano_stscd_nunique,acqic_stocn_nunique,acqic_hcefg_nunique,bacno_stscd_nunique,acqic_flbmk_nunique,acqic_loctm_nunique,acqic_bacno_nunique,acqic_stscd_nunique
1521787,6862,162489,116168,16158,5,0,0,0,0,0,5,0,0,77950,457,59060,0,0,102,0,1,2,2,1,3,2,2,56166,30627,2
1521788,6862,162489,116168,16158,5,0,0,0,0,0,5,0,0,79549,457,59060,0,0,102,0,1,2,2,1,3,2,2,56166,30627,2
1521789,6862,162489,116168,16158,5,0,0,0,0,0,5,0,0,60355,457,59060,0,0,102,0,1,2,2,1,3,2,2,56166,30627,2
1521790,6862,162489,116168,16158,5,0,0,0,0,0,5,0,0,60296,457,59060,0,0,102,0,1,2,2,1,3,2,2,56166,30627,2
1521791,6862,162489,116168,16158,5,0,0,0,0,0,5,0,0,77933,457,59060,0,0,102,0,1,2,2,1,3,2,2,56166,30627,2


# save

In [22]:
train.to_csv("/content/drive/My Drive/玉山人工智慧比賽數據/train_特徵工程完.csv")
test.to_csv("/content/drive/My Drive/玉山人工智慧比賽數據/test_特徵工程完.csv")
print('save done!')

save done!
